In [2]:
import pandas as pd
import numpy as np
import datetime
import ast
import psycopg2 

from sqlalchemy import create_engine
from sqlalchemy import create_engine, Table, MetaData
import sqlalchemy.types

In [ ]:
# Indices details as ticker, market...
credentials = ast.literal_eval(open(r"C:\Users\pablo\OneDrive - unizar.es\Python Investigación\credentials.txt", "r").read())
sql_engine = create_engine(credentials['sqlalchemy'][0]+ 'IndexSustainabilityLevel')
conn = psycopg2.connect(dbname='IndexSustainabilityLevel', user=credentials['psycopg2'][0], 
                    password=credentials['psycopg2'][1], host=credentials['psycopg2'][2], port =credentials['psycopg2'][3])
cur = conn.cursor()

In [41]:
equity_indices_detail=pd.read_excel(r'C:\Users\pablo\OneDrive - unizar.es\Artículo 1\4.1. Descarga capitalización y sectores de las empresas ESG Reuters (7.469)\EquityIndicesDetail.xlsx')
equity_indices_detail.to_sql('EquityIndexDetails', sql_engine, if_exists="replace", index=False)

In [ ]:
# indices constituents to database
# change numeric date to datetime
def obtain_date(numeric_date):    
    numeric_date = str(numeric_date)
    numeric_date = datetime.datetime(int(numeric_date[0:4]),int(numeric_date[4:6]),1)
    return numeric_date

equity_indices_details = pd.read_sql('EquityIndexDetails', sql_engine)
for row in range(0, len(equity_indices_details)):
    index_ticker = equity_indices_details.iat[row,1]
    # we read each listado with the information of the index
    df_index = pd.read_excel(r'C:\Users\pablo\OneDrive - unizar.es\Artículo 1\4.1. Descarga capitalización y sectores de las empresas ESG Reuters (7.469)\1. Listados\Listados Índices\%s.xlsx' % index_ticker)
    df_index['IndexTicker'] = index_ticker # We add a column with the ticker of the index
    df_index = df_index.loc[:, ['IndexTicker', 'ISIN', 'Fecha Numerica', 'Type', 'NAME', 'WTIDX']] # change order of columns
    df_index.columns = ['IndexTicker', 'IsinCode', 'MonthlyDate', 'DsType', 'DsName', 'CompanyWeight'] # change the name
    df_index['MonthlyDate'] = df_index['MonthlyDate'].apply(obtain_date) # numeric date to datetime
    # we append the columns to the databse
    df_index.to_sql('EquityIndicesConstituents', sql_engine, index=False, 
                    if_exists="append", chunksize=100000, method='multi' )
    print(index_ticker)
    

In [ ]:
monthly_scores = pd.read_excel(r"C:\Users\pablo\OneDrive - unizar.es\Artículo 1\4.1. Descarga capitalización y sectores de las empresas ESG Reuters (7.469)\Excel central con  tablas de trabajo.xlsm",
                          usecols ="B:D, R:U, Y" )

all_scores.columns = ['MonthlyDate', 'DsName', 'IsinCode', 'MarketValueUsd', 'EsgScore', 'CombinedScore',
       'ControversiesScore', 'WcNation']

all_scores.to_sql('CompaniesMonthlyData', sql_engine, chunksize=100000, method='multi', index=False)

In [ ]:
geo_classification = pd.read_excel(r"C:\Users\pablo\OneDrive - unizar.es\Artículo 1\4.1. Descarga capitalización y sectores de las empresas ESG Reuters (7.469)\Excel central con  tablas de trabajo.xlsm",
                          sheet_name ="Info Títulos" )

In [4]:
# Indices details as ticker, market...
credentials = ast.literal_eval(open(r"C:\Users\pablo\OneDrive - unizar.es\Python Investigación\credentials.txt", "r").read())
sql_engine = create_engine(credentials['sqlalchemy'][0]+ 'FinancialResearch')

In [31]:
df = pd.read_sql("""  
SELECT info.isin_code, cont.year, cont.year, cont.end_fiscal_period, cont.datastream_id, cont.return_on_assets,  cont.return_on_equity, 
cont.long_term_debt / NULLIF(cont.total_assets,0) AS long_debt_to_assets,
cont.total_liabilities /  NULLIF(cont.total_assets,0) AS total_liabilities_to_assets,
cont.additions_fixed_assets / NULLIF(cont.total_assets,0) AS additions_fixed_assets_to_assets
FROM market.yearly_accounting AS cont
INNER JOIN esg.companies_esg_refinitiv AS info
ON info.datastream_id = cont.datastream_id
""", sql_engine)

In [34]:
credentials = ast.literal_eval(open(r"C:\Users\pablo\OneDrive - unizar.es\Python Investigación\credentials.txt", "r").read())
sql_engine = create_engine(credentials['sqlalchemy'][0]+ 'IndexSustainabilityLevel')
df.dropna().to_sql('yearly_accounting', sql_engine, if_exists='replace', chunksize=100000, method='multi', index=False)